# Processing Drug-Target Interaction Data

This notebook covers:
- Converting merged DTI data into an `h5torch` dataset
- Splitting the dataset (stratified) into train/val/test in two settings: random split, and cold-start split
- Computing embeddings from foundation models
    - Drugs: `MMELON` (graph, image, text), and RDKit fingerprints
    - Targets: `NT`, `ESM`, and ESPF fingerprints

In [1]:
import os

# Set the working directory to the root of the project
os.chdir("/home/robsyc/Desktop/thesis/MB-VAE-DTI")

In [2]:
import pandas as pd
import numpy as np
from typing import Tuple, List, Dict, Any

df = pd.read_csv("data/processed/data.csv")
df

,Drug_ID,Drug_InChIKey,Drug_SMILES,Target_ID,Target_UniProt_ID,Target_Gene_name,Target_RefSeq_ID,Target_AA,Target_DNA,Y,Y_pKd,Y_pKi,Y_KIBA,in_DAVIS,in_BindingDB_Kd,in_BindingDB_Ki,in_Metz,in_KIBA
0,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000001,Q2M2I8,AAK1,NM_014911,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,ATGAAGAAGTTTTTCGACTCCCGGCGAGAGCAGGGCGGCTCTGGCC...,False,4.999996,NaN,NaN,True,True,False,False,False
1,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000002,P00519,ABL1,NM_005157,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,AACCTTTTCGTTGCACTGTATGATTTTGTGGCCAGTGGAGATAACA...,True,8.920819,NaN,NaN,True,False,False,False,False
2,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000003,Q04771,ACVR1,NM_001105,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,ATGGTAGATGGAGTGATGATTCTTCCTGTGCTTATCATGATTGCTC...,False,4.999996,NaN,NaN,True,True,False,False,False
3,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000004,P36896,ACVR1B,NM_004302,MAESAGASSFFPLVVLLLAGSGGSGPRGVQALLCACTSCLQANYTC...,ATGGCGGAGTCGGCCGGAGCCTCCTCCTTCTTCCCCCTTGTTGTCC...,False,4.999996,NaN,NaN,True,True,False,False,False
4,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000005,P27037,ACVR2A,NM_001278579,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...,ATGGGAGCTGCTGCAAAGTTGGCGTTTGCCGTCTTTCTTATCTCCT...,False,4.999996,NaN,NaN,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396464,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T001711,P41279,MAP3K8,NM_001244134,MEYMSTGSDNKEEIDLLIKHLNVSDVIDIMENLYASEEPAVYEPSL...,ATGGAGTACATGAGCACTGGAAGTGACAATAAAGAAGAGATTGATT...,True,NaN,NaN,13.70206,False,False,False,False,True
396465,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T000031,Q13554,CAMK2B,NM_001220,MATTVTCTRFTDEYQLYEDIGKGAFSVVRRCVKLCTGHEYAAKIIN...,ATGGCCACCACGGTGACCTGCACCCGCTTCACCGACGAGTACCAGC...,False,NaN,NaN,10.49794,False,False,False,False,True
396466,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T000032,Q13557,CAMK2D,NM_001221,MASTTTCTRFTDEYQLFEELGKGAFSVVRRCMKIPTGQEYAAKIIN...,ATGGCTTCGACCACAACCTGCACCAGGTTCACGGACGAGTATCAGC...,False,NaN,NaN,10.49794,False,False,False,False,True
396467,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T000280,Q16539,MAPK14,NM_001315,MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKT...,ATGTCTCAGGAGAGGCCCACGTTCTACCGGCAGGAGCTGAACAAGA...,False,NaN,NaN,10.49794,False,False,False,False,True


In [3]:
from mb_vae_dti.processing import add_split_cols, create_h5torch

df_split = add_split_cols(df)
df_split

00:21:06 - INFO - Old pandas version detected. Patching DataFrame.map to DataFrame.applymap


,Drug_ID,Drug_InChIKey,Drug_SMILES,Target_ID,Target_UniProt_ID,Target_Gene_name,Target_RefSeq_ID,Target_AA,Target_DNA,Y,Y_pKd,Y_pKi,Y_KIBA,in_DAVIS,in_BindingDB_Kd,in_BindingDB_Ki,in_Metz,in_KIBA,split_rand,split_cold
0,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000001,Q2M2I8,AAK1,NM_014911,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,ATGAAGAAGTTTTTCGACTCCCGGCGAGAGCAGGGCGGCTCTGGCC...,False,4.999996,NaN,NaN,True,True,False,False,False,train,train
1,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000002,P00519,ABL1,NM_005157,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,AACCTTTTCGTTGCACTGTATGATTTTGTGGCCAGTGGAGATAACA...,True,8.920819,NaN,NaN,True,False,False,False,False,train,train
2,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000003,Q04771,ACVR1,NM_001105,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...,ATGGTAGATGGAGTGATGATTCTTCCTGTGCTTATCATGATTGCTC...,False,4.999996,NaN,NaN,True,True,False,False,False,train,train
3,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000004,P36896,ACVR1B,NM_004302,MAESAGASSFFPLVVLLLAGSGGSGPRGVQALLCACTSCLQANYTC...,ATGGCGGAGTCGGCCGGAGCCTCCTCCTTCTTCCCCCTTGTTGTCC...,False,4.999996,NaN,NaN,True,True,False,False,False,test,train
4,D000001,KTUFNOKKBVMGRW-UHFFFAOYSA-N,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,T000005,P27037,ACVR2A,NM_001278579,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...,ATGGGAGCTGCTGCAAAGTTGGCGTTTGCCGTCTTTCTTATCTCCT...,False,4.999996,NaN,NaN,True,True,False,False,False,train,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396464,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T001711,P41279,MAP3K8,NM_001244134,MEYMSTGSDNKEEIDLLIKHLNVSDVIDIMENLYASEEPAVYEPSL...,ATGGAGTACATGAGCACTGGAAGTGACAATAAAGAAGAGATTGATT...,True,NaN,NaN,13.70206,False,False,False,False,True,train,train
396465,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T000031,Q13554,CAMK2B,NM_001220,MATTVTCTRFTDEYQLYEDIGKGAFSVVRRCVKLCTGHEYAAKIIN...,ATGGCCACCACGGTGACCTGCACCCGCTTCACCGACGAGTACCAGC...,False,NaN,NaN,10.49794,False,False,False,False,True,train,train
396466,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T000032,Q13557,CAMK2D,NM_001221,MASTTTCTRFTDEYQLFEELGKGAFSVVRRCMKIPTGQEYAAKIIN...,ATGGCTTCGACCACAACCTGCACCAGGTTCACGGACGAGTATCAGC...,False,NaN,NaN,10.49794,False,False,False,False,True,train,train
396467,D162026,NGCNZPVYWSEEJK-UHFFFAOYSA-N,CCCc1nc[nH]c1CNc1cc(Cl)c2ncc(C#N)c(Nc3ccc(F)c(...,T000280,Q16539,MAPK14,NM_001315,MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKT...,ATGTCTCAGGAGAGGCCCACGTTCTACCGGCAGGAGCTGAACAAGA...,False,NaN,NaN,10.49794,False,False,False,False,True,train,train


In [4]:
df_split["split_rand"].value_counts() / len(df_split)

split_rand
train    0.799977
test     0.100043
valid    0.099980
Name: count, dtype: float64

In [12]:
create_h5torch(df_split, output_filename="data.h5torch")

Creating h5torch file from dataframe with 396469 rows...
Found 149962 unique drugs and 2047 unique targets
Creating central interaction matrix of shape (149962, 2047) with 396469 observed interactions
Y values include: ['Y', 'Y_pKd', 'Y_pKi', 'Y_KIBA', 'in_DAVIS', 'in_BindingDB_Kd', 'in_BindingDB_Ki', 'in_Metz', 'in_KIBA']
Registering drug features...
Registering target features...
Registering split information as unstructured data...
Created h5torch file at data/processed/data.h5torch

Usage example:

# Load dataset with 'coo' sampling for training split
train_dataset = h5torch.Dataset(
    "data/processed/data.h5torch",
    sampling="coo",
    subset=("unstructured/split_cold", "train"),
    in_memory=True,
)


In [13]:
import h5torch

file = h5torch.File("data/processed/data.h5torch", "r")
print(file["central/indices"][:].shape)      # (2, 396469)
print(file["central/indices"][:][0].max())   # 0 - 149961
print(file["0"]["Drug_ID"][:].shape)         # (149962,)
print(file["central/indices"][:][1].max())   # 2046
print(file["1"]["Target_ID"][:].shape)       # (2047,)

print(len(file["0"]["Drug_SMILES"][:]))      # 149962
print(len(df_split["Drug_SMILES"].unique())) # 149962

(2, 396469)
149961
(149962,)
2046
(2047,)
149962
149962


In [7]:
39664 / 396469

0.10004313073657714

In [14]:
# Load dataset with 'coo' sampling for training split
train_dataset = h5torch.Dataset(
    "data/processed/data.h5torch",
    sampling="coo",
    subset=("unstructured/split_cold", "test"),
    in_memory=True,
)

len(train_dataset), train_dataset[100]

TypeError: data type "[('Y', '?'), ('Y_pKd', '<f8'), ('Y_pKi', '<f8'), ('Y_KIBA', '<f8'), ('in_DAVIS', '?'), ('in_BindingDB_Kd', '?'), ('in_BindingDB_Ki', '?'), ('in_Metz', '?'), ('in_KIBA', '?')]" not understood

In [25]:
df_split[(df_split["Drug_ID"] == "D000028") & (df_split["Target_ID"] == "T000101")]

,Drug_ID,Drug_InChIKey,Drug_SMILES,Target_ID,Target_UniProt_ID,Target_Gene_name,Target_RefSeq_ID,Target_AA,Target_DNA,Y,Y_pKd,Y_pKi,Y_KIBA,in_DAVIS,in_BindingDB_Kd,in_BindingDB_Ki,in_Metz,in_KIBA,split_rand,split_cold
7741,D000028,XZXHXSATPCNXJR-ZIADKAODSA-N,COC(=O)c1ccc2c(c1)NC(=O)C2=C(Nc1ccc(N(C)C(=O)C...,T000101,P42685,FRK,NM_002031,MSNICQRLWEYLEPYLPCLSTEADKSTVIENPGALCSPQSQRHGHY...,ATGAGCAACATCTGTCAGAGGCTCTGGGAGTACCTAGAACCCTATC...,False,5.56862,NaN,NaN,True,False,False,False,False,train,test
